In [6]:
import pymongo
import pandas as pd
import random
import json
from os.path import join,dirname
import os
from pymongo import MongoClient
import random
from datetime import date, timedelta, datetime
DATEFORMAT = "%Y-%m-%d"

client = MongoClient('localhost', 27017)
db = client['aion']
collection = db['employee']

ROOT_DIR = os.path.abspath(os.curdir)
filename =join(ROOT_DIR, 'data/employee_mock_data.json')
if filename:
    with open(filename, 'r') as f:
        employees = json.load(f)
print(employees[0:2])
# bias salary by age and by gender
employee_list = []
employee_pay = []
employee_gender_lst = []
employee_age_lst = []
for key,employee in enumerate(employees):
    employee_list.append(employee['name'])
    employee_gender_lst.append(employee['gender'].lower())
    if employee['hourly_rate'] < 0:
        employee['hourly_rate'] = 20
    if employee['gender'].lower() == 'female':
        employee['hourly_rate'] *= random.choice([1.3, 1.6, 1.4, 1.1])
    dob = datetime.strptime(employee['dob'],DATEFORMAT)
    employee['dob'] = datetime(dob.year,dob.month,dob.day,0,0,0)
    age = int(round((datetime.today() - dob).days / 365,0))
    employee_age_lst.append(age)
    employee['hourly_rate'] += round((datetime.today() - employee['dob']).days/900,2)
    employee_pay.append(round(employee['hourly_rate'],2))
print(employees[0:2])
employee_pay_dct = dict(zip(employee_list,employee_pay))
employee_gender_dct = dict(zip(employee_list,employee_gender_lst))
employee_age_dct = dict(zip(employee_list,employee_age_lst))
print(employee_pay)


[{'id': 1, 'name': 'Cindra Serotsky', 'department': 'Marketing', 'title': 'Senior Cost Accountant', 'gender': 'Female', 'dob': '1993-06-09', 'hourly_rate': 14}, {'id': 2, 'name': 'Nataline Marples', 'department': 'Business Development', 'title': 'Actuary', 'gender': 'Female', 'dob': '1996-09-07', 'hourly_rate': 20}]
[{'id': 1, 'name': 'Cindra Serotsky', 'department': 'Marketing', 'title': 'Senior Cost Accountant', 'gender': 'Female', 'dob': datetime.datetime(1993, 6, 9, 0, 0), 'hourly_rate': 32.92}, {'id': 2, 'name': 'Nataline Marples', 'department': 'Business Development', 'title': 'Actuary', 'gender': 'Female', 'dob': datetime.datetime(1996, 9, 7, 0, 0), 'hourly_rate': 37.2}]
[32.92, 37.2, 26.69, 45.56, 37.57, 44.63, 39.46, 31.05, 30.53, 39.95, 54.36, 35.79, 30.43, 38.62, 29.71, 43.13, 38.78, 41.22, 31.0, 46.5, 44.7, 38.95, 56.46, 32.34, 44.8, 52.86, 41.07, 36.91, 46.32, 27.43, 35.46, 29.73, 24.73, 18.79, 53.41, 45.88, 39.07, 36.63, 39.89, 38.02, 26.99, 32.49, 37.22, 47.13, 41.44, 43

In [36]:
import numpy as np
from scipy.stats import gamma
import random


def determine_owner(dct1,type):
    owner = random.choice(employee_list)
    gender = employee_gender_dct[owner].lower()
    age = employee_age_dct[owner]
    dct1[type+'_owner'] = owner
    dct1[type+'_owner_gender'] = gender
    dct1[type+'_owner_age'] = age
    return dct1
     
 
def determine_delays(dct2,sp,ep,type):
    dct2[type+'_startdate_proposed'] = sp
    dct2[type+'_enddate_proposed'] = ep

    if type == 'project':
        a = 20
    elif type == 'milestone':
        a = 11
    elif type == 'task':
        a = 2
    
    delay = int(gamma.rvs(a=a, size=1))
    gender = dct2[type+'_owner_gender']
    age = dct2[type+'_owner_age']
    if gender == 'female':
        delay *= .7
    delay = delay * (65 - age)*.02 # older is better
    if delay < 0:
        delay = 0
    sa = sp + timedelta(days=delay)
    txt = type + '_startdate_actual'
    dct2[txt] = sa
    # project_enddate delay
    txt = type + '_enddate_actual'
    
    if type == 'project':
        if s == 'closed':
            delay = int(gamma.rvs(a=a, size=1))
            delay *= random.choice([-1,1,1])
            if gender == 'female':
                delay *= .4
            if delay < 0:
                delay = delay * age*1.4 # older is better
            else:
                delay = delay * (65-age)*.03 # older is better
            
            if delay < 0:
                delay *= -1
            ea = ep + timedelta(days=delay)
            dct2[txt] = ea
            if type in ['project','milestone']:
                dct2[type+'_end_delay'] = (ea - ep).days 
            else:
                dct2[type+'_end_delay'] = (ea - ep).days * 8
            dct2[type+'_duration'] = (dct2[type+'_enddate_actual'] - dct2[type+'_startdate_actual']).days
            if type in ['task']:
                dct2[type+'_duration'] *= 8

        else:
            ea = datetime(2020,1,1,0,0,0)
            dct2[txt] = ea
            dct2[type+'_duration'] = (dct2[type+'_enddate_proposed'] - dct2[type+'_startdate_proposed']).days
            if type in ['task']:
                dct2[type+'_duration'] *= 8
            if type in ['project','milestone']:
                dct2[type+'_end_delay'] = (ea - ep).days 
            else:
                dct2[type+'_end_delay'] = (ea - ep).days * 8

    else:
        delay = int(gamma.rvs(a=a, size=1))
        delay *= random.choice([-1,1,1])
        if gender == 'female':
            delay *= .4
        if delay < 0:
            delay = delay * age*1.4 # older is better
        else:
            delay = delay * (65-age)*.03 # older is better
        
        ea = ep + timedelta(days=delay)

        if delay < 0:
            delay *= -1
        dct2[txt] = ea
        if type in ['project','milestone']:
            dct2[type+'_end_delay'] = (ea - ep).days 
        else:
            dct2[type+'_end_delay'] = (ea - ep).days * 8
        dct2[type+'_duration'] = (dct2[type+'_enddate_actual'] - dct2[type+'_startdate_actual']).days
        if type in ['task']:
            dct2[type+'_duration'] *= 8

    if type in ['project','milestone']:
        dct2[type+'_start_delay'] = (sa - sp).days 
    else:
        dct2[type+'_start_delay'] = (sa - sp).days * 8
    
    
    return dct2
    



In [38]:
# fill projects
collection = db['project_type']
dct_type = {
    'type':['research','innovation','audit','reconciliation','conference','manufacturing',
            'construction']
}
#df_type = pd.DataFrame.from_dict(dct_type)
#data_json = json.loads(df_type.to_json(orient='records'))
#collection.insert_many(data_json)

types = dct_type['type']

collection_workers = db['project_task']
tasks_list = []
project_lists = []
composite_lst = []
for idx in range(1,6000):
    dct = {}
    composite={}
    type = 'project'
    dct = determine_owner(dct,type)
    # generate project type
    name = 'Project '+str(idx)
    dct['project_name'] = name
    dct['type'] = random.choice(types)
    s = random.choice(['closed','closed','open'])
    # generate dates
    first_date = datetime(2013,1,1,0,0,0)
    days = (datetime.today() - first_date).days
    sp = first_date + timedelta(days=random.choice(list(range(0,days))))
    
    dct['project_startdate_proposed'] = sp
    proj_len = int(gamma.rvs(a=80, size=1))
    if proj_len < 0:
        proj_len *= -1
    ep = sp + timedelta(days=proj_len)
    dct['project_enddate_proposed'] = ep
    
    s = 'closed'
    if ep > datetime.today():
        s = 'open'
    dct['status'] = s

    dct = determine_delays(dct,sp,ep,type)
    
    project_lists.append(dct)
    
    # --- milestones
    # schedule miles stones bases on reducing difference
    # ---------
    proj_rem = (ep - sp).days
    mep = sp
    mcounter=0
    while mep < ep:
        mcounter += 1
        dct_m = {}
        dct_m['milestone_name'] = 'Milestone ' + str(mcounter)
        msp = mep + timedelta(days=random.randint(0,proj_rem)) # proposed start occurs after end of last mileston
        proj_rem = (ep - mep).days #  update proj remain
        mep = msp + timedelta(int(gamma.rvs(a=proj_rem, size=1)))
        if mep > ep:
            mep = ep
        typem = 'milestone'
        dct_m = determine_owner(dct_m,typem)
        dct_m = determine_delays(dct_m,msp,mep,typem)
        
        ####################### tasks
        emps = int(gamma.rvs(a=20, size=1))
        for emp in range(0,emps):
            composite = {}
            dct_tasks = {}
            dct_t = {}
            dct_tasks['project'] = name
            task_len = proj_len * 8 *.5# hours

            start = dct_m['milestone_startdate_actual'] + timedelta(days=random.choice(list(range(0,int(proj_len*.8)))))
            dct_tasks['start'] = start
            end = start + timedelta(hours= int(gamma.rvs(a=task_len, size=1)))
            if end > dct_m['milestone_enddate_actual']:
                end = dct_m['milestone_enddate_actual']
            
            dct_tasks['end'] = end

            typet = 'task'
            dct_t = determine_owner(dct_t,typet)
            dct_t = determine_delays(dct_t,start,end,typet)
            duration = (dct_tasks['end'] - dct_tasks['start']).days*8
            composite = {
                'project':name,
                'project_owner':dct['project_owner'],
                'project_owner_gender': dct['project_owner_gender'],
                'project_owner_age':dct['project_owner_age'],
                'type':dct['type'],
                'status':dct['status'],
                'project_startdate_proposed':dct['project_startdate_proposed'],
                'project_startdate_actual':dct['project_startdate_actual'],
                'project_enddate_proposed':dct['project_enddate_proposed'],
                'project_enddate_actual':dct['project_enddate_actual'],
                'project_duration':dct['project_duration'],
                'project_start_delay':dct['project_start_delay'],
                'project_end_delay':dct['project_end_delay'],

                
                # milestone
                'milestone_name':dct_m['milestone_name'],
                'milestone_owner' : dct_m['milestone_owner'],
                'milestone_owner_gender': dct_m['milestone_owner_gender'],
                'milestone_owner_age':dct_m['milestone_owner_age'],
                'milestone_startdate_proposed':dct_m['milestone_startdate_proposed'],
                'milestone_startdate_actual':dct_m['milestone_startdate_actual'],
                'milestone_enddate_proposed':dct_m['milestone_enddate_proposed'],
                'milestone_enddate_actual':dct_m['milestone_enddate_actual'],
                'milestone_duration':dct_m['milestone_duration'],
                'milestone_start_delay':dct_m['milestone_start_delay'],
                'milestone_end_delay':dct_m['milestone_end_delay'],
                
                'task_name':'Task '+str(emp),
                'task_owner':dct_t['task_owner'],
                'task_owner_gender': employee_gender_dct[dct_t['task_owner']].lower(),
                'task_owner_age': employee_gender_dct[dct_t['task_owner']].lower(),
                'task_startdate_proposed':dct_t['task_startdate_proposed'],
                'task_enddate_proposed':dct_t['task_enddate_proposed'],
                'task_duration':dct_t['task_duration'],
                'task_start_delay':dct_t['task_start_delay'],
                'task_end_delay':dct_t['task_end_delay'],
                
                
                'remuneration':round(employee_pay_dct[dct_t['task_owner']]*abs(dct_t['task_duration']),2),
                
                
            }
            tasks_list.append(dct_tasks)
            composite_lst.append(composite)

#collection_project = db['project']
#collection_project.insert_many(project_lists)
#collection_workers.insert_many(tasks_list)
collection_composite = db['project_composite']
collection_composite.insert_many(composite_lst)
